In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc


In [2]:
train = pd.read_csv('../../data/train_groups.csv')
test  = pd.read_csv('../../data/test_groups.csv')

In [3]:
test['is_train'] = False
train['is_train'] = True


In [6]:
data_x = pd.concat(( train.drop('target', 1), test))

In [5]:
title = pd.read_csv('../../data/docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')

## Preprocess Text Labels

In [ ]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re
mystem = Mystem()
russian_stopwords = stopwords.words("russian")
alph = r'[<>\%\(\)abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ/#;:!{}_|\-\?.=&<>@\[\]""]'

information = []
for i in title.title:
    try:
        tokens = mystem.lemmatize(i.lower())
        tokens
        tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
        text = " ".join(tokens)
        text = re.sub(alph, '', text)
        text = re.sub(r'[\s+\d+\n]', ' ', text)
        information.append(text)
        
    except:
        information.append(np.nan)

In [ ]:
title['information'] = information

In [4]:
title = pd.read_csv('nem_title.csv')

In [7]:
data = data_x.merge(title, how = 'left', on = 'doc_id')
data = data.replace(np.nan, 'noinformation')

In [8]:
title = pd.read_csv('nem_title.csv')

## Make Hash

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer


vectorizer = HashingVectorizer(n_features=2**14, dtype = 'float32' ,ngram_range=(1, 3) )
X = vectorizer.fit_transform(data.information )

In [104]:
X.shape

(28317, 16384)

## Make DataFrame

In [9]:
# data_tfidf.shape
data_hash = pd.DataFrame(X.toarray())
# data_hash.to_csv('data_hash')

In [10]:
data_hash['doc_id'] = data.doc_id
data_hash['pair_id'] = data.pair_id
data_hash['group_id'] = data.group_id
data_hash['is_train'] = data.is_train


### train and test

In [14]:
new_train = data_hash[data_hash.is_train]
new_train['target'] = train.target

new_test = data_hash[~data_hash.is_train]

### Group by target

In [ ]:
true_vector_mean = new_train.drop(['pair_id', 'is_train','group_id','doc_id'], 1).groupby('terget').agg('mean')

### Group by group_id


In [14]:
true_group_vector_mean = new_train.drop(['pair_id', 'is_train','target'],1).groupby('group_id').agg('mean')

In [26]:
true_group_vector_mean.to_csv('vectors_of_groups.csv', index = False)

### Find the same group in test

In [22]:
test_group_vector_mean = new_test.drop(['pair_id', 'is_train'],1).groupby('group_id').agg('mean')

In [25]:
test_group_vector_mean.to_csv('test_vectors_of_groups.csv', index = False)

In [16]:
from scipy.spatial.distance import cosine


### find distanses for mean vectors of groups 

In [17]:
cos_mean = []

for i in tqdm(train_vectors_of_groups.index):
    q = train_vectors_of_groups.loc[i]
    for j in new_train[new_train.group_id == i + 1].index:
        cos_mean.append(cosine(q, new_train.drop(['doc_id','pair_id','group_id','is_train','target'],1).loc[j]))
    

  0%|          | 0/129 [00:00<?, ?it/s]/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 129/129 [54:04<00:00, 23.84s/it]


In [28]:
train1['cos_mean'] = cos_mean
train1.head()

/home/kirill/playground/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group,cos_mean
0,1,1,15731,0,True,0.975970,0.0,0.506909
1,2,1,14829,0,True,0.939010,0.0,0.585735
2,3,1,15764,0,True,0.965664,0.0,0.773604
3,4,1,17669,0,True,0.990659,0.0,0.950442
4,5,1,14852,0,True,0.950488,0.0,0.670281


In [30]:
train1.corr()

,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group,cos_mean
pair_id,1.000000,0.999900,-0.121821,0.041701,NaN,-0.021072,0.103803,-0.070021
group_id,0.999900,1.000000,-0.122197,0.041418,NaN,-0.020639,0.102131,-0.069258
doc_id,-0.121821,-0.122197,1.000000,-0.017996,NaN,-0.049112,-0.081367,-0.065106
target,0.041701,0.041418,-0.017996,1.000000,NaN,0.142582,0.254589,-0.254728
is_train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_true_hash,-0.021072,-0.020639,-0.049112,0.142582,NaN,1.000000,0.084109,0.152689
train_group,0.103803,0.102131,-0.081367,0.254589,NaN,0.084109,1.000000,-0.199284
cos_mean,-0.070021,-0.069258,-0.065106,-0.254728,NaN,0.152689,-0.199284,1.000000


In [19]:
cos_mean2 = []
for i in tqdm(test_vectors_of_groups.index):
    q = test_vectors_of_groups.loc[i]
    for j in new_test[new_test.group_id == i + 130].index:
        cos_mean2.append(cosine(q, new_test.drop(['doc_id','pair_id','group_id','is_train'],1).loc[j]))

  0%|          | 0/180 [00:00<?, ?it/s]/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 180/180 [1:48:16<00:00, 35.95s/it]


In [20]:

test['cos_mean'] = cos_mean2
test.head()

,pair_id,group_id,doc_id,is_train,cos_mean
0,11691,130,6710,False,0.553029
1,11692,130,4030,False,0.806924
2,11693,130,5561,False,0.641318
3,11694,130,4055,False,0.629449
4,11695,130,4247,False,0.752520


In [70]:
new_train.train_group = new_train.train_group.replace(np.nan, 0)


In [57]:
new_test.train_group = new_test.train_group.replace(np.nan, 0)

In [74]:
new_train.to_csv('new_train.csv', index = False)
new_test.to_csv('new_test.csv', index = False)

In [73]:
new_train.head()

,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group
0,1,1,15731,0,True,0.975970,0.0
1,2,1,14829,0,True,0.939010,0.0
2,3,1,15764,0,True,0.965664,0.0
3,4,1,17669,0,True,0.990659,0.0
4,5,1,14852,0,True,0.950488,0.0


## find distances for mean vector of all true data

In [25]:
def make_cos(vector, new_test = new_test):
    cos = []
    for i in  new_test.index:
        cos.append(cosine(vector, new_test.loc[i]))
    return cos

In [35]:
train_mean_true = make_cos(true_false_vector_mean.loc[0], new_test = new_train.drop('target', 1))
test_mean_true = make_cos(true_false_vector_mean.loc[0])


/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [37]:
train['mean_true_hash'] = train_mean_true
test['mean_true_hash'] = test_mean_true


### Data preporation

In [69]:
doc_to_title = {}
with open('../../data/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [78]:
traingroups_titledata = {}
for i in range(len(train1)):
    new_doc = train1.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
    
testgroups_titledata = {}
for i in range(len(test1)):
    new_doc = test1.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [79]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 15) (11690,) (11690,)


# Build Model

### Firstly XGBoostClassifier

In [39]:
from xgboost import XGBClassifier 
from sklearn.metrics import f1_score

In [40]:
xgb = XGBClassifier(n_estimators=1200,  random_state = 0, nthread = 4, learning_rate = 0.01,
                        max_depth=4)

In [46]:
train.shape

(11690, 6)

In [77]:
train1['target'] = train.target

In [75]:
# train1.drop(['target','train_group','is_train'], 1, inplace = True)
test1.drop(['train_group','is_train'], 1, inplace= True)

In [64]:
groups = train1.group_id.unique()
X = train1[train1['group_id'].isin(groups[:int(len(groups)*4/6)])]
X_val = train1[train1['group_id'].isin(groups[int(len(groups)*4/6):])]
y = y_train[train['group_id'].isin(groups[:int(len(groups)*4/6)])]
y_val = y_train[train['group_id'].isin(groups[int(len(groups)*4/6):])]

In [65]:
xgb.fit(X, y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=1200,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [66]:
f1_score(y_val, xgb.predict(X_val))

0.4677880038083148

In [ ]:
test = data_hash.drop('information', 1)[~data_hash.is_train].drop('is_train', 1)
pred = xgb.predict(test)

In [18]:
n_estimators = [400, 800, 1500, 2000, 4000]
max_depth = [3,5,8,12]
best params = np.zeros(2)
for i in n_estimators:
    for j in max_depth:
        xgb = XGBClassifier(n_estimators=i,  random_state = 0, nthread = 4, learning_rate = 0.01,
                        max_depth=j)
        xgb.fit(X,y)
        score = f1_score(y_val, xgb.predict(X_val))
        if score > best_score:
            best_score = score
            best_params = [i, j]
        
    

[0]	validation_0-logloss:0.690953
[1]	validation_0-logloss:0.688804
[2]	validation_0-logloss:0.690046
[3]	validation_0-logloss:0.687965
[4]	validation_0-logloss:0.685913
[5]	validation_0-logloss:0.683903
[6]	validation_0-logloss:0.685161
[7]	validation_0-logloss:0.683365
[8]	validation_0-logloss:0.681455
[9]	validation_0-logloss:0.682753
[10]	validation_0-logloss:0.680885
[11]	validation_0-logloss:0.6792
[12]	validation_0-logloss:0.677393
[13]	validation_0-logloss:0.678669
[14]	validation_0-logloss:0.677051
[15]	validation_0-logloss:0.675349
[16]	validation_0-logloss:0.676655
[17]	validation_0-logloss:0.674954
[18]	validation_0-logloss:0.676264
[19]	validation_0-logloss:0.67463
[20]	validation_0-logloss:0.673139
[21]	validation_0-logloss:0.671549
[22]	validation_0-logloss:0.672863
[23]	validation_0-logloss:0.671307
[24]	validation_0-logloss:0.672618
[25]	validation_0-logloss:0.671085
[26]	validation_0-logloss:0.66971
[27]	validation_0-logloss:0.66824
[28]	validation_0-logloss:0.666911


[232]	validation_0-logloss:0.667371
[233]	validation_0-logloss:0.667767
[234]	validation_0-logloss:0.669079
[235]	validation_0-logloss:0.668711
[236]	validation_0-logloss:0.668471
[237]	validation_0-logloss:0.668175
[238]	validation_0-logloss:0.66787
[239]	validation_0-logloss:0.66752
[240]	validation_0-logloss:0.668831
[241]	validation_0-logloss:0.668559
[242]	validation_0-logloss:0.668586
[243]	validation_0-logloss:0.669893
[244]	validation_0-logloss:0.669798
[245]	validation_0-logloss:0.669512
[246]	validation_0-logloss:0.669243
[247]	validation_0-logloss:0.668955
[248]	validation_0-logloss:0.668621
[249]	validation_0-logloss:0.668533
[250]	validation_0-logloss:0.669863
[251]	validation_0-logloss:0.670756
[252]	validation_0-logloss:0.670473
[253]	validation_0-logloss:0.670279
[254]	validation_0-logloss:0.669953
[255]	validation_0-logloss:0.669897
[256]	validation_0-logloss:0.669575
[257]	validation_0-logloss:0.670468
[258]	validation_0-logloss:0.670228
[259]	validation_0-logloss:0.6

[461]	validation_0-logloss:0.685466
[462]	validation_0-logloss:0.686046
[463]	validation_0-logloss:0.685914
[464]	validation_0-logloss:0.686007
[465]	validation_0-logloss:0.685891
[466]	validation_0-logloss:0.685929
[467]	validation_0-logloss:0.685808
[468]	validation_0-logloss:0.685716
[469]	validation_0-logloss:0.685601
[470]	validation_0-logloss:0.686919
[471]	validation_0-logloss:0.686809
[472]	validation_0-logloss:0.686655
[473]	validation_0-logloss:0.686541
[474]	validation_0-logloss:0.686419
[475]	validation_0-logloss:0.686306
[476]	validation_0-logloss:0.685932
[477]	validation_0-logloss:0.685835
[478]	validation_0-logloss:0.687104
[479]	validation_0-logloss:0.686999
[480]	validation_0-logloss:0.6869
[481]	validation_0-logloss:0.686797
[482]	validation_0-logloss:0.686689
[483]	validation_0-logloss:0.687193
[484]	validation_0-logloss:0.687074
[485]	validation_0-logloss:0.686702
[486]	validation_0-logloss:0.687967
[487]	validation_0-logloss:0.687859
[488]	validation_0-logloss:0.6

[690]	validation_0-logloss:0.694633
[691]	validation_0-logloss:0.694549
[692]	validation_0-logloss:0.694479
[693]	validation_0-logloss:0.694423
[694]	validation_0-logloss:0.694348
[695]	validation_0-logloss:0.694125
[696]	validation_0-logloss:0.694052
[697]	validation_0-logloss:0.693986
[698]	validation_0-logloss:0.694061
[699]	validation_0-logloss:0.693994
[700]	validation_0-logloss:0.693919
[701]	validation_0-logloss:0.693987
[702]	validation_0-logloss:0.693915
[703]	validation_0-logloss:0.69381
[704]	validation_0-logloss:0.693731
[705]	validation_0-logloss:0.693649
[706]	validation_0-logloss:0.693623
[707]	validation_0-logloss:0.693567
[708]	validation_0-logloss:0.69348
[709]	validation_0-logloss:0.693556
[710]	validation_0-logloss:0.694676
[711]	validation_0-logloss:0.694621
[712]	validation_0-logloss:0.694553
[713]	validation_0-logloss:0.694227
[714]	validation_0-logloss:0.695107
[715]	validation_0-logloss:0.695028
[716]	validation_0-logloss:0.694956
[717]	validation_0-logloss:0.6

[919]	validation_0-logloss:0.696834
[920]	validation_0-logloss:0.696772
[921]	validation_0-logloss:0.6967
[922]	validation_0-logloss:0.696652
[923]	validation_0-logloss:0.696598
[924]	validation_0-logloss:0.696535
[925]	validation_0-logloss:0.696476
[926]	validation_0-logloss:0.696427
[927]	validation_0-logloss:0.696341
[928]	validation_0-logloss:0.696297
[929]	validation_0-logloss:0.696209
[930]	validation_0-logloss:0.696133
[931]	validation_0-logloss:0.696038
[932]	validation_0-logloss:0.695966
[933]	validation_0-logloss:0.695907
[934]	validation_0-logloss:0.69649
[935]	validation_0-logloss:0.696378
[936]	validation_0-logloss:0.696109
[937]	validation_0-logloss:0.696055
[938]	validation_0-logloss:0.696022
[939]	validation_0-logloss:0.695968
[940]	validation_0-logloss:0.69595
[941]	validation_0-logloss:0.695872
[942]	validation_0-logloss:0.695817
[943]	validation_0-logloss:0.695772
[944]	validation_0-logloss:0.69572
[945]	validation_0-logloss:0.695674
[946]	validation_0-logloss:0.6956

[1144]	validation_0-logloss:0.691842
[1145]	validation_0-logloss:0.691567
[1146]	validation_0-logloss:0.691521
[1147]	validation_0-logloss:0.691476
[1148]	validation_0-logloss:0.691448
[1149]	validation_0-logloss:0.691403
[1150]	validation_0-logloss:0.691395
[1151]	validation_0-logloss:0.691328
[1152]	validation_0-logloss:0.691287
[1153]	validation_0-logloss:0.691069
[1154]	validation_0-logloss:0.691027
[1155]	validation_0-logloss:0.690756
[1156]	validation_0-logloss:0.690699
[1157]	validation_0-logloss:0.690652
[1158]	validation_0-logloss:0.690607
[1159]	validation_0-logloss:0.690614
[1160]	validation_0-logloss:0.690554
[1161]	validation_0-logloss:0.690525
[1162]	validation_0-logloss:0.690464
[1163]	validation_0-logloss:0.690263
[1164]	validation_0-logloss:0.690248
[1165]	validation_0-logloss:0.690186
[1166]	validation_0-logloss:0.690138
[1167]	validation_0-logloss:0.689929
[1168]	validation_0-logloss:0.690455
[1169]	validation_0-logloss:0.690413
[1170]	validation_0-logloss:0.690367
[

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=1200,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
pred = xgb/pred()

In [21]:
test = pd.read_csv('test_tfidf.csv', dtype = 'float32')
test.columns = X.columns
pred = xgb.predict(test)
Counter(pred)

Counter({0: 2425, 1: 14621})

In [19]:
from collections import Counter

In [20]:
# data_tfidf = pd.read_csv('all_tfidf.csv', dtype = 'float32')